In [0]:
WITH onlybaseentities as (
  SELECT
    child
  FROM
    `na-cdp-bu-finance-dev`.curated.EPMParentChildHierarchy
  WHERE
    `IsBaseRecord` = 1
    AND adjacencypath like 'Entity|Shared_Mbr_Entity_Structure|Management|BU_NA|%'
),
branchnames as (
  SELECT
    backendname,
    alias
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmdimensionmetadata`
),
usd_data as (
  SELECT
    TRUNC(TO_DATE(CAST(dateint AS STRING), 'yyyyMMdd'), 'MONTH') as `Calendar Date`,
    branch.alias as entity,
    CASE
      WHEN costtype = 'COS DIRECT' THEN 'Direct Value '
      WHEN costtype = 'COS INDIRECT' THEN 'Indirect Value '
      WHEN costtype = 'G&A' THEN 'Indirect Value '
      WHEN costtype = 'SELLING' THEN 'Indirect Value '
    END as costtypecategory,
    CASE
      WHEN Breakdown2Id = 'TKE101' THEN 'NI'
      WHEN Breakdown2Id = 'TKE102' THEN 'MOD'
      WHEN Breakdown2Id = 'TKE1031' THEN 'SERV'
      WHEN Breakdown2Id = 'TKE1032' THEN 'REPAIR'
    END AS LOB,
    usd.AmountAnnual as Value
  FROM
    `na-cdp-bu-hr-dev`.curated.usdamount as usd
      JOIN onlybaseentities as child
        on child.child = usd.epmbranch
      LEFT JOIN branchnames as branch
        on branch.backendname = usd.epmbranch
  WHERE
    CostType IN ('COS DIRECT', 'COS INDIRECT', 'SELLING', 'G&A')
    and Breakdown2Id IN ('TKE101', 'TKE102', 'TKE1031', 'TKE1032')
    and usd.AmountAnnual <> 0
),
ttls_entity as (
  SELECT
    `Calendar Date`,
    entity as Entity,
    LOB,
    costtypecategory as Account,
    ROUND(sum(Value), 3) as Value
  FROM
    usd_data
  GROUP BY
    ALL
),
ttls_buna as (
  SELECT
    `Calendar Date`,
    'BU North America' as Entity,
    LOB,
    costtypecategory as Account,
    ROUND(sum(Value), 3) as Value
  FROM
    usd_data
  GROUP BY
    ALL
)
SELECT
  *,
  'Actuals' as Scenario,
  'Headcount USD' as Source
FROM
  (
    SELECT
      *
    FROM
      ttls_entity
    UNION ALL
    SELECT
      *
    FROM
      ttls_buna
  )